In [8]:
import pandas as pd
import numpy as np
import os, glob,re

from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

from IPython.core.display import HTML

import torch
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

In [ ]:
!pip install transformers
!pip install pytorch_lightning

     |████████████████████████████████| 2.2MB 12.6MB/s 
     |████████████████████████████████| 870kB 39.8MB/s 
     |████████████████████████████████| 3.3MB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b579d67a8cf6ba6f6ed6efaf2e4c6049c31525766dc6294c3b061ec8735c483c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import transformers
import pytorch_lightning as pl

pl.seed_everything(13)

In [2]:
df = pd.read_csv(Path(PATH) / 'train.csv')
df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


Look for text in Json


In [4]:
df1 = pd.read_json(Path(PATH) /'train' /'d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json')
df1

,section_title,text
0,What is this study about?,This study used data from the National Educati...
1,Features of Dual Enrollment Programs,Dual enrollment programs allow high school stu...
2,WWC Single Study Review,"What did the study find?\nThe study reported, ..."
3,WWC Rating,The research described in this report meets WW...
4,Intervention group,The intervention group was comprised of those ...
5,Comparison group,The comparison group was comprised of NELS par...
6,Outcomes and measurement,There were two outcomes examined in the study:...
7,Support for implementation,The study was based on secondary data from a n...
8,Reason for review,This study was identified for review by receiv...
9,Degree attainment,Any college degree attainment The study author...


for BERT not useble very length text, base model 512 tokens

In [17]:
print(df1.text.apply(lambda x: len(x.split())).describe())
print()
print('All length text in json: ', len(df1.text.str.cat(sep = '\n').split()))

count     18.000000
mean      98.000000
std       95.736157
min       10.000000
25%       27.750000
50%       41.500000
75%      165.000000
max      298.000000
Name: text, dtype: float64

All length text in json:  1764


Nice len in d0fa7568-7d8e-4db9-870f-f9c6f668c17b

min = 10 and max 298 by title, and we check all len(all text) is eq = 1764


Okay lets see all json length param

In [30]:
def see_length(file: Path):
    # make conf interval for length
    tmp_df = pd.read_json(file)
    tmp_stat = tmp_df.text.apply(lambda x: len(x.split())).describe().to_dict()
    len_text = len(tmp_df.text.str.cat(sep = '\n').split())
    id = file.stem
    return {
        'id':id,
        'min': tmp_stat['min'],
        'mean': tmp_stat['mean'],
        'max': tmp_stat['max'],
        'len_txt': len_text
        }

list_train_json = list(Path(PATH).glob('train/*.json'))

tmp = []
# time by colab is 1h
for file in tqdm(list_train_json):
    tmp_dict = see_length(file)
    tmp.append(
        pd.DataFrame(tmp_dict,index=[0])
        )
length_df = pd.concat(tmp).reset_index(drop=True)
length_df.head()

100%|██████████| 14316/14316 [59:22<00:00,  4.02it/s]


,index,id,min,mean,max,len_txt
0,0,f0249680-2273-4b1a-8a0a-2e8bbf9ec11a,0.0,316.541667,1197.0,15194
1,0,ef4b7964-b7dc-4570-804e-74cc7b804c61,42.0,427.266667,985.0,6409
2,0,ec938f38-37e7-4a35-87bd-e59ffa02d52e,76.0,318.214286,532.0,4455
3,0,f042f6d8-c940-4cda-9b96-75cb95db4ef8,0.0,378.200000,705.0,3782
4,0,ed1a995e-63f9-4542-b515-bcaab0ed4295,0.0,380.789474,2738.0,7235


In [31]:
length_df.mean(axis = 0)

index         0.000000
min         553.132858
mean        924.293620
max        1864.406259
len_txt    7049.041771
dtype: float64

In [32]:
length_df['min'].value_counts().sort_values(ascending=False)

0.0         9052
15.0          79
26.0          77
2.0           74
1.0           70
            ... 
328235.0       1
564.0          1
1356.0         1
522.0          1
1014.0         1
Name: min, Length: 632, dtype: int64

In [33]:
length_df[length_df['min'] < 100].mean(axis = 0)

index         0.000000
min          10.655568
mean        374.762448
max        1352.718951
len_txt    6855.988687
dtype: float64

we see more len is 0 let see what it is

In [192]:
pd.DataFrame(['cas covid 19 antiviral candidate compounds dataset',
              'cas covid 19 antiviral candidate compounds dataset',
              'cas cov antiviral candidate compounds dataset'
])[0].str.contains('covid').sum()

2

In [193]:
def have_digit(txt: str) -> bool:
    return any(char.isdigit() for char in txt)

reusult = df.cleaned_label.apply(lambda x: have_digit(x))

In [200]:
for i in df[reusult].cleaned_label.values:
    if 'covid' not in i:
        print(i)
        break

characterizing health associated risks and your baseline disease in sars cov 2 charybdis 


In [202]:
def the_smallest(txt: str)->int:
    tmp = list(len(x) for x in txt.split() if not x.isdigit())
    return tmp[np.argmin(tmp)]

In [203]:
df['the_smallest'] = df.cleaned_label.apply(lambda x: the_smallest(x))

In [206]:
df[df.the_smallest == 1].head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,the_smallest
412,237b2194-9c15-4e33-b9b7-e9a0ab37f499,A Process Oriented Areal Interpolation Techniq...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
413,c210f9ed-b69e-4bd6-aa60-d94de4efe7e1,Voluntary Restoration: Mitigation's Silent Par...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
414,a9f6ce1b-829b-4fdf-9c0c-cb04261e21ad,The Multi-Resolution Land Characteristics (MRL...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
415,bf02931b-7872-4c63-9837-51479e37b5d3,Evaluation of Airborne Lidar Elevation Surface...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
416,aac0d000-4569-41fe-b071-118d93d6068a,Change in Land Cover along the Lower Columbia ...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1


In [208]:
def the_smallest_txt_label(txt: str)->int:
    for x in txt.split():
        if (not x.isdigit()) and (len(txt.split()) == 1):
            return len(x)

df['len_label'] =df.cleaned_label.apply(lambda x: the_smallest_txt_label(x))

In [209]:
df['len_label'].value_counts()

4.0    3673
7.0     280
5.0       2
Name: len_label, dtype: int64

In [211]:
a = 'LITERATURE REVIEW Page adakdkah Figure'
re.sub(r'(\bCHAPTER|Page|Figure\b)', ' ', a)

'LITERATURE REVIEW   adakdkah  '

In [237]:
def clean_text(txt: str)-> str:
    return re.sub('[^A-Za-z]+', ' ', str(txt).lower()).strip()

def clean_text_add_digit(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())


def len_title(txt: str)->str:
    """
    String is p p p Result:  nan
    String is Andi Result:  Andi
    String is pr Result:  nan
    String is aa aa aa Result:  nan
    String is cas cov Result:  nan
    String is cas c Result:  nan
    String is c a s c o v as Result:  nan
    String is a 0 gv) Result:  nan
    String is ft Result:  nan
    String is All all Result:  nan
    String is Alaa 2 Result:  Alaa 2
    String is A Aa2a Result:  A Aa2a
    String is cas Result:  nan
    String is covantiviral Result:  covantiviral
    String is candidate compounds dataset Result:  candidate compounds dataset

    if one word len >= 4 return word
    else 
        if less and one of words >= 4 words
    else nan
    
    """
    if txt is np.nan:
        return txt
    txt = txt.split()
    # len('Andi'.split()) == 1
    if (len(txt) == 1) and (len(txt[0])>= 4):
        # one word
        txt = txt[0]
    else:
        len_each_word = list(len(i) for i in txt)
        max_len_txt = txt[np.argmax(len_each_word)]
        if len(max_len_txt)>= 4:
            if len_each_word == len(txt):
                txt = np.nan
            else:              
                txt = ' '.join(txt)
        else:
            txt = np.nan
    return txt

def change_len(txt: str)-> str: 
    if txt is np.nan:
        return txt   
    return ' '.join(list(i for i in txt.split() if len(txt.split()) >= 10))


def clear_data(txt: str, check = False):
    """
    txt : path to json file
    check: bool if true return dataframe else save json
    
    
    """
    tmp_df = pd.read_json(txt)
    """
    a = 'CHAPTER 2. LITERATURE REVIEW Page'
    re.sub(r'(\bCHAPTER|Page|Figure\b)', ' ', a) ->   2. LITERATURE REVIEW  
    """
    tmp_df.section_title = tmp_df.section_title.apply(lambda x:
                                                      re.sub(r'(\bCHAPTER|Page\b)', ' ', x).strip())
    """
    2. delete 
        C. Career activities after obtaining a doctora...
        IV. Changes to the 1998 SED Marital Status
        B. Policy recommendtions for the utilization .
        1A.
        1.
        GH12.
    """

    tmp_df.section_title = tmp_df.section_title.apply(lambda x:
                                                      re.sub('^[A-Z0-9]+(?=\.)', ' ', x).strip())
    """
    if covid in line use clear with digits save else skip digit
    only in covid label have digit
    """
    if (tmp_df.section_title.str.contains('covid', flags=re.IGNORECASE).sum() != 0) or \
       (tmp_df.text.str.contains('covid', flags=re.IGNORECASE).sum() != 0):
        # print('cov')
        #covid more digits
        tmp_df.section_title = tmp_df.section_title.apply(lambda x: clean_text_add_digit(x))
        tmp_df.text = tmp_df.text.apply(lambda x: clean_text_add_digit(x))
    else:
         # clear digit
        tmp_df.section_title = tmp_df.section_title.apply(lambda x: clean_text(x))
        tmp_df.text = tmp_df.text.apply(lambda x: clean_text(x))

    #chek length title and if not correct droped
    #find blank line
    tmp_df.section_title.replace('', np.nan, inplace=True)
    tmp_df.text.replace('', np.nan, inplace=True)
    # idea for title if sum len words in title == len title drop, [1,1,1] == 'a t u'  
    tmp_df.section_title = tmp_df.section_title.apply(lambda x: len_title(x))
    tmp_df.text = tmp_df.text.apply(lambda x: change_len(x))

    tmp_df.section_title.replace('', np.nan, inplace=True)
    tmp_df.text.replace('', np.nan, inplace=True)

    url = '(https?://|w{3}\.)(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    tel = '(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'

    tmp_df.dropna(inplace =True)

    for rgx in [url, tel]:
        tmp_df.text = tmp_df.text.apply(lambda x:
                                    re.sub(rgx, ' ', x))
        
    if check is False:
        #save json??        
        path_save = Path(PATH) /'data_train_v1'/ txt.name
        tmp_df.reset_index(drop=True).to_json(path_save, orient="columns")
    else:
        return tmp_df.reset_index(drop=True)

In [238]:
f = 'b3498176-8832-4033-aea6-b5ea85ea04c4.json'
f = 'e9057a7a-ea41-4b40-88bf-3bfb8354c807.json'
f = 'f0249680-2273-4b1a-8a0a-2e8bbf9ec11a.json'
f = 'c6e74b14-6da5-43bf-9723-dcce7c2124ff.json'
f = '9e7a78df-4757-40d8-b347-fe6578f0e4ad.json'
f = '755506a3-1516-4c8b-8fc0-ca10027aed61.json'
f = '6bfda128-a0c0-41ba-96a8-ec62731124d9.json'


f = Path(PATH) /'train' / f

cld = clear_data(f, check=True)
cld

,section_title,text
0,careers of korean ph ds with degrees of foreign,countries and the hrd policy of the highly ski...
1,overview,due to the development of information and comm...
2,main results of the research,a the overall number of foreign doctoral holde...
3,career activities after obtaining a doctorate ...,at the time of receiving a doctorate degree hi...
4,basic directions,the policies should be made with a basic under...
5,policy recommendations for the utilization of ...,strategies to attract more doctoral recipients...


In [239]:
df7 = pd.read_json(Path(PATH) /'train' / f)
df7

,section_title,text
0,Careers of Korean Ph.Ds with Degrees of Foreign,Countries and the HRD Policy of the Highly Ski...
1,Overview,Due to the development of information and comm...
2,Main Results of the Research,A. The overall number of foreign doctoral hold...
3,C. Career activities after obtaining a doctora...,"At the time of receiving a doctorate degree, h..."
4,Policy Implications for Utilizing High Quality...,Who Earned a Doctorate Degree Abroad
5,A. Basic directions,The policies should be made with a basic under...
6,B. Policy recommendations for the utilization ...,1) Strategies to attract more doctoral recipie...


In [241]:
clear_data(f)
f = Path(PATH) /'data_train_v1'/ f.name
print(f)
pd.read_json(f)

/content/drive/MyDrive/Coleridge_Initiative/input/data_train_v1/6bfda128-a0c0-41ba-96a8-ec62731124d9.json


,section_title,text
0,careers of korean ph ds with degrees of foreign,countries and the hrd policy of the highly ski...
1,overview,due to the development of information and comm...
2,main results of the research,a the overall number of foreign doctoral holde...
3,career activities after obtaining a doctorate ...,at the time of receiving a doctorate degree hi...
4,basic directions,the policies should be made with a basic under...
5,policy recommendations for the utilization of ...,strategies to attract more doctoral recipients...
